## Loading Images

In [2]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image dimensions
img_width, img_height = 64, 64

# Directory paths
train_dir = 'dataset/Gambo/Train'
test_dir = 'dataset/Gambo/Test'

# Create an instance of ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1.0/255.0)  # Normalize pixel values to [0, 1]

# Load 1000 images from each class (Normal and Reversal) for training
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,  # You can choose a batch size; this will load 32 images at a time
    class_mode='binary',  # Binary classification: Normal (0), Reversal (1)
    classes=['Normal', 'Reversal'],  # Specify class subdirectories
    shuffle=True,
    subset=None,
)

# Load 1000 images from each class (Normal and Reversal) for testing
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    classes=['Normal', 'Reversal'],
    shuffle=False,  # No need to shuffle during testing
    subset=None,
)


Found 86115 images belonging to 2 classes.
Found 37439 images belonging to 2 classes.


## Training the ANN Model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Build the ANN model
model = Sequential()

# Flatten the image from 64x64x3 (RGB) to a single vector
model.add(Flatten(input_shape=(img_width, img_height, 3)))

# Add dense layers
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

# Output layer (for binary classification: Normal/Reversal)
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


c:\Users\baaqa\miniconda3\envs\tf-cpu\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 12288)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,572,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,581,313 (6.03 MB)

 Trainable params: 1,581,313 (6.03 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=12000 // train_generator.batch_size,  # Total 2000 images (1000 Normal, 1000 Reversal)
    epochs=5,  # You can adjust the number of epochs based on performance
    validation_data=test_generator,
    validation_steps=20000 // test_generator.batch_size  # Total 2000 images (1000 Normal, 1000 Reversal)
)


Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 104s 277ms/step - accuracy: 0.9712 - loss: 0.0860 - val_accuracy: 0.8752 - val_loss: 0.3650
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 97s 259ms/step - accuracy: 0.9768 - loss: 0.0635 - val_accuracy: 0.7326 - val_loss: 1.1696
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 100s 266ms/step - accuracy: 0.9721 - loss: 0.0786 - val_accuracy: 0.9083 - val_loss: 0.2510
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 72s 193ms/step - accuracy: 0.9762 - loss: 0.0694 - val_accuracy: 0.7948 - val_loss: 0.6882
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 64s 172ms/step - accuracy: 0.9724 - loss: 0.0792 - val_accuracy: 0.9293 - val_loss: 0.2089


## Evaluation

In [16]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {accuracy * 100:.2f}%')


1170/1170 ━━━━━━━━━━━━━━━━━━━━ 55s 47ms/step - accuracy: 0.9302 - loss: 0.2351
Test Accuracy: 87.03%


## Making Predictions

In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img) / 255.0  # Normalize the image
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Predict the class
    prediction = model.predict(img_array)
    if prediction < 0.5:
        print("Normal")
    else:
        print("Reversal")

# Test on a new image
for i in range(900,921):
    predict_image(f"1_{i}.png", model)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Reversal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Reversal


## Save the model

In [18]:
# Save the trained model
model.save('pattern_classifier_20000.h5')
